# Desafio: algoritmo de Shor para fatorar 21 (a = 2)

Este notebook replica o fluxo do tutorial principal para o caso \(N = 21\) e \(a = 2\), construindo os operadores de multiplicação modular, o circuito de estimativa de fase (QPE) e o pós-processamento clássico para recuperar os fatores de 21.


In [ ]:
import os
from fractions import Fraction
from math import floor, gcd, log

import numpy as np
from dotenv import load_dotenv
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit.circuit.library import QFT, UnitaryGate
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler

load_dotenv()

# Parâmetros do desafio
a = 2
N = 21
num_target = floor(log(N - 1, 2)) + 1  # número de qubits-alvo para representar N
num_control = 2 * num_target           # precisão da QPE (regra do tutorial)

print(f"Usando N={N}, a={a}, num_target={num_target}, num_control={num_control}")


In [ ]:
def a2kmodN(a: int, k: int, N: int) -> int:
    """Calcula a^{2^k} (mod N) por quadrados sucessivos."""
    for _ in range(k):
        a = int(np.mod(a**2, N))
    return a


def mod_mult_gate(b: int, N: int) -> QuantumCircuit:
    """Porta de multiplicação modular |x> -> |b*x mod N>.

    Preferimos sintetizar via perm2circuit (permutações) para reduzir profundidade
    em relação ao UnitaryGate. Mantém identidade nos estados acima de N.
    """
    if gcd(b, N) > 1:
        raise ValueError(f"gcd({b},{N}) > 1, escolha b coprimo de N")

    n_qubits = floor(log(N - 1, 2)) + 1
    dim = 2**n_qubits

    # Permutação explícita
    perm = list(range(dim))
    for x in range(N):
        perm[x] = (b * x) % N
    # estados >= N já estão como identidade

    # Tenta sintetizar como circuito de permutação
    try:
        from qiskit.synthesis.permutation import perm2circuit

        qc = perm2circuit(perm)
        qc.name = f"M{b}"
        return qc
    except Exception:
        pass

    # Fallback: UnitaryGate (pode ficar mais profundo)
    mat = np.zeros((dim, dim), dtype=complex)
    for x in range(dim):
        y = perm[x]
        mat[y, x] = 1
    gate = UnitaryGate(mat, label=f"M{b}mod{N}")
    qc = QuantumCircuit(n_qubits, name=f"M{b}")
    qc.append(gate, range(n_qubits))
    return qc


# Pré-computa os b = a^{2^k} (mod N) usados nos controles
b_list = [a2kmodN(a, k, N) for k in range(num_control)]
print("b_list (a^{2^k} mod N):", b_list)

# Cria cache de portas M_b e controladas
Mb_cache = {}
controlled_Mb_cache = {}
for b in set(b_list):
    Mb = mod_mult_gate(b, N)
    Mb_cache[b] = Mb
    controlled_Mb_cache[b] = Mb.to_gate().control(1)

print("Portas únicas construídas:", list(Mb_cache.keys()))


In [ ]:
# Constrói o circuito de determinação de ordem usando QPE
ctrl = QuantumRegister(num_control, "ctrl")
tgt = QuantumRegister(num_target, "tgt")
cl = ClassicalRegister(num_control, "c")
circ = QuantumCircuit(ctrl, tgt, cl)

# Inicializa |1> no registrador alvo
circ.x(tgt[0])

# Hadamards nos controles
circ.h(ctrl)

# Exponenciação modular controlada (para cada qubit de controle)
for k in range(num_control):
    b = b_list[k]
    gate = controlled_Mb_cache[b]
    circ.append(gate, [ctrl[k], *tgt])

# QFT inversa nos controles
circ.compose(QFT(num_control, do_swaps=True, approximation_degree=0).inverse(), ctrl, inplace=True)

# Medição
circ.measure(ctrl, cl)

print(circ)



In [ ]:
ibm_api_key = os.getenv("IBM_API_KEY")
if not ibm_api_key:
    raise ValueError("Defina IBM_API_KEY no .env ou nas variáveis de ambiente.")

ibm_instance = os.getenv("QISKIT_IBM_INSTANCE")
if not ibm_instance:
    raise ValueError("Defina QISKIT_IBM_INSTANCE no .env ou nas variáveis de ambiente.")

service = QiskitRuntimeService(
    channel="ibm_quantum_platform",
    token=ibm_api_key,
    instance=ibm_instance,
)
backend_name = os.getenv("QISKIT_IBM_BACKEND", "ibm_marrakesh")
backend = service.backend(backend_name)

# Integração alinhada ao notebook principal: transpila para o backend e executa no Sampler
pm = generate_preset_pass_manager(optimization_level=2, backend=backend)
pub = pm.run(circ)

sampler = Sampler(backend)
shots = 1024
job = sampler.run([pub], shots=shots)
result = job.result()
shots_used = getattr(result[0], "metadata", {}).get("shots", shots)
pub_data = result[0].data

print(f"Backend: {backend_name}")
print("Resultado bruto (data):", pub_data)



In [ ]:
# Extrai contagens do SamplerV2
pub_data = result[0].data
counts = None

# 1) API direta (preferida)
try:
    counts = pub_data.get_counts("c")
except Exception:
    counts = None

# 2) BitArray direto (DataBin.c)
if counts is None:
    bitarr = None
    try:
        if hasattr(pub_data, "c"):
            bitarr = pub_data.c
        elif hasattr(pub_data, "__getitem__") and "c" in pub_data:
            bitarr = pub_data["c"]
    except Exception:
        bitarr = None
    if bitarr is not None:
        try:
            if hasattr(bitarr, "get_counts"):
                counts = bitarr.get_counts()
            elif hasattr(bitarr, "frequencies"):
                raw = bitarr.frequencies()
                counts = {}
                for k, v in raw.items():
                    if isinstance(k, (int, np.integer)):
                        bit_lbl = format(int(k), f"0{num_control}b")
                    else:
                        bit_lbl = str(k).zfill(num_control)
                    counts[bit_lbl] = int(v)
        except Exception:
            counts = None

# 3) Quasi-probabilidades (get_value)
if counts is None:
    try:
        quasi = pub_data.get_value()
        counts = {}
        shots_used = getattr(result[0], "metadata", {}).get("shots", shots)
        for k, prob in quasi.items():
            try:
                bit_int = int(k) if not isinstance(k, str) else int(k, 2)
            except Exception:
                continue
            counts[format(bit_int, f"0{num_control}b")] = int(round(float(prob) * shots_used))
    except Exception:
        counts = None

if counts is None:
    raise ValueError(f"Não foi possível extrair contagens de pub_data. Tipo: {type(pub_data)}, attrs: {dir(pub_data)}")

print("Contagens (normalizadas):", counts)

In [ ]:
# Pós-processamento para estimar r e obter fatores
rows = []
factors = set()

for bitstring, shots in counts.items():
    # Ajusta endianness: Qiskit retorna little-endian
    decimal = int(bitstring[::-1], 2)
    phase = decimal / (2**num_control)
    frac = Fraction(phase).limit_denominator(N)
    r_candidate = frac.denominator

    # Tenta extrair fatores se r for par
    if r_candidate % 2 == 0 and r_candidate > 0:
        x = pow(a, r_candidate // 2, N)
        f1 = gcd(x - 1, N)
        f2 = gcd(x + 1, N)
        if 1 < f1 < N:
            factors.add(f1)
        if 1 < f2 < N:
            factors.add(f2)

    rows.append((bitstring, shots, phase, f"{frac.numerator}/{frac.denominator}", r_candidate))

# Ordena por contagem decrescente
rows = sorted(rows, key=lambda x: x[1], reverse=True)

print("Resultados ordenados (bitstring, shots, fase, frac, r):")
for row in rows[:10]:
    print(row)

print("Fatores encontrados:", factors if factors else "Nenhum fator não trivial encontrado nesta rodada")
